In [373]:
import joblib
import pandas as pd
import numpy as np
# Preprocessing
import preprocessor as p
import re
import string
import contractions
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords
# Model
from sklearn import model_selection, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
# Pipeline
from sklearn.pipeline import make_pipeline
from joblib import dump

In [374]:
t1_classifier = joblib.load("../out/final-test-wstops/" + 't1_classifier.joblib.z')

C:\Users\johna\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\johna\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\johna\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\johna\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your

In [375]:
test_df = pd.read_csv("../res/test_final.csv")

In [376]:
def expand_contractions(s):
    expanded = []
    for word in s.split():
        expanded.append(contractions.fix(word))
    return(' '.join(expanded))

test_df["text"] = [expand_contractions(entry) for entry in test_df['text']]
# Remove punctuation marks
test_df["text"] = [entry.translate(str.maketrans('', '', string.punctuation)) for entry in test_df["text"]]
# Tokenization
test_df['text']= [word_tokenize(entry) for entry in test_df['text']]

In [377]:
# TODO: Experiment if NOT removing stop words will improve model accuracy

# Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(test_df['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    test_df.loc[index,'text_final'] = str(Final_words)

In [379]:
#test_df['text'] = [','.join(map(str, l)) for l in test_df['text']]
bracketed_words = set()
# Regex pattern to match anything that is eclosed by angled brackets
pattern = re.compile("\<(.*?)\>")
# Get all unique bracketed tokens
for e in test_df["text"]:
    for t in e:
        if ">" in t and "<" in t:
            bracketed_words.add(t)
# Remove all bracketed tokens in post_tokens
test_df["text"] = [list(filter(lambda token: token not in bracketed_words, entry)) for entry in test_df["text"]]
test_df.head()

,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None,text_final
0,"[its, about, time, blacks, realised, the, wome...",1,0,0,1,0,1,0,0,0,"['time', 'black', 'realise', 'woman', 'start',..."
1,"[the, islamic, and, sharia, controlled, middle...",0,0,1,1,0,1,0,0,0,"['islamic', 'sharia', 'control', 'middle', 'ea..."
2,"[so, now, you, are, calling, yourself, whoever...",0,0,1,1,0,0,0,0,1,"['call', 'whoever', 'agree', 'severely', 'reta..."
3,"[a, million, thumbs, up, we, need, museums, of...",0,1,0,1,1,0,0,0,0,"['million', 'thumb', 'need', 'museum', 'victim..."
4,"[these, your, kike, bosses, too, feldman, is, ...",1,0,0,1,0,1,0,0,0,"['kike', 'boss', 'feldman', 'cernovichs', 'bos..."


In [380]:
y_t1_classifier = test_df[["HATE","NOT","OFFN"]]
x_test = test_df[["text_final"]]

In [381]:
x_t1_classifier = t1_classifier.predict(x_test)
print("SVM Accuracy Score -> ",accuracy_score(x_t1_classifier, y_t1_classifier)*100)

ValueError: Found input variables with inconsistent numbers of samples: [1, 2885]

In [ ]:
train_Y = Encoder.fit_transform(train_y)
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(subtask1,train_y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(subtask1)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, test_y)*100)